
# Projet 2

In [4]:
# import des modules usuels
import numpy as np
import pandas as pd

# options d'affichage
pd.set_option("display.min_rows", 16)

In [5]:
# chargement et traitement des données
geo = pd.read_csv("correspondance-code-insee-code-postal.csv",
                   sep=';',
                   usecols=range(11),
                   index_col="Code INSEE")
geo = geo.sort_index()
geo[['Latitude', 'Longitude']] = geo["geo_point_2d"].str.extract("(.*), (.*)").astype(float)
geo.head()

,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude
Code INSEE,,,,,,,,,,,,
01001,01400,L'ABERGEMENT-CLEMENCIAT,AIN,RHONE-ALPES,Commune simple,242.0,1565.0,0.8,"46.1534255214, 4.92611354223","{""type"": ""Polygon"", ""coordinates"": [[[4.926273...",46.153426,4.926114
01002,01640,L'ABERGEMENT-DE-VAREY,AIN,RHONE-ALPES,Commune simple,483.0,912.0,0.2,"46.0091878776, 5.42801696363","{""type"": ""Polygon"", ""coordinates"": [[[5.430089...",46.009188,5.428017
01004,01500,AMBERIEU-EN-BUGEY,AIN,RHONE-ALPES,Chef-lieu canton,379.0,2448.0,13.4,"45.9608475114, 5.3729257777","{""type"": ""Polygon"", ""coordinates"": [[[5.386190...",45.960848,5.372926
01005,01330,AMBERIEUX-EN-DOMBES,AIN,RHONE-ALPES,Commune simple,290.0,1605.0,1.6,"45.9961799872, 4.91227250796","{""type"": ""Polygon"", ""coordinates"": [[[4.895580...",45.996180,4.912273
01006,01300,AMBLEON,AIN,RHONE-ALPES,Commune simple,589.0,602.0,0.1,"45.7494989044, 5.59432017366","{""type"": ""Polygon"", ""coordinates"": [[[5.614854...",45.749499,5.594320


**Projet maison n° 1**

- Ajoutez une colonne 'CP Ville' avec le Code postal + un espace + et le nom de la Ville.
- Ecrivez une fonction qui détermine la commune la plus proche d'un point à partir de sa latitude et sa longitude.
- Ajoutez une fonction de conversion pour pouvoir utiliser la première fonction avec un GPS (degrés, minutes, secondes).

In [6]:
#Ajout de la colonne CP Ville

geo["CP Ville"] = geo["Code Postal"] + " " + geo["Commune"]

In [7]:
#Fonction de recherche d'une valeur la plus proche dans une array
def find_nearest(array, value):
    array = np.asarray(array)
    y = (np.abs(array - value)).argmin()
    return array[y]

In [8]:
# fonction recherche de ville
def ville(lat, long):
    latitude = geo["Latitude"]
    longitude = geo["Longitude"]
    
    geo_ville = geo.loc[np.isclose(geo["Latitude"], find_nearest(latitude, lat), rtol=1e-02) & np.isclose(geo["Longitude"], find_nearest(longitude, long), rtol=1e-02)]
    geo_ville_lat = geo_ville["Latitude"]
    geo_ville_long = geo_ville["Longitude"]
    y = []
    for i,j in zip(geo_ville_lat, geo_ville_long):
        y.append(np.sqrt(((lat - i)**2)+((long - j)**2)))


    insee_ville = geo_ville_lat.index[y.index(min(y))]

    return "La ville la plus proche est : " + geo["CP Ville"].loc[insee_ville]
    

In [9]:
# on applique la fonction à une coordonnée tirée au hasard
np.random.seed(0)
a, b = 41.5, 51.1  # latitude min et max de la France métropolitaine
lat = (b - a) * np.random.random() + a
a, b = -5.1, 9.5  # longitude min et max de la France métropolitaine
long = (b - a) * np.random.random() + a

ville(lat, long)

'La ville la plus proche est : 71330 BOSJEAN'

In [10]:
# conversion degrés, minutes, secondes => décimal
def dms2dec(deg, mn, sec):
    return deg + (mn/60) + (sec/3600)

In [11]:
# à partir de coordonnées GPS précises
ville(dms2dec(48, 42, 52), dms2dec(2, 14, 45))

'La ville la plus proche est : 91120 PALAISEAU'

**Projet maison n° 2**

La colonne "geo_shape" comporte des chaines de catactères au format JSON. Elles représentent les formes géométriques des communes qui sont soit des polygones soit composées de plusieurs polygones.

- Utiliser la librairie Python json pour parser la colonne "geo_shape".
- Donner le décompte des valeurs accédées avec la clé "type".
- Donner le décompte des longueurs des listes accédées avec la clé "coordinates".
- Quelle commune est la plus complexe géométriquement ?
- Quelle commune est la seconde la plus complexe géométriquement ?
- Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?
- Pour ces villes vérifier que le premier polygone contient bien le second (enclave). NB : installer la librairie shapely, utiliser la classe Polygon de shapely.geometry. Sur Windows shapely peut nécessiter d'installer la dll "geos_c.dll" dans le répertoire "Library/bin" de votre environnement Python.

In [129]:
#Utiliser la librairie Python json pour parser la colonne "geo_shape".

import json

geo["geo_shape_json"] = geo["geo_shape"].apply(lambda x: json.loads(x))

In [130]:
#Donner le décompte des valeurs accédées avec la clé "type".

geo["type"] = geo["geo_shape_json"].apply(lambda x: x["type"])
geo["type"].value_counts()

Polygon         36670
MultiPolygon       72
Name: type, dtype: int64

In [133]:
#Donner le décompte des longueurs des listes accédées avec la clé "coordinates".

import itertools

geo["coordinates"] = geo["geo_shape_json"].apply(lambda x: x["coordinates"])
geo
def merged(x):
    res = list(itertools.chain(*x))
    return res

geo["merged_coordinates"] = geo["coordinates"].apply(lambda x: len(merged(x)))
geo["merged_coordinates"] = geo["merged_coordinates"]
#geo

In [134]:
#Quelle commune est la plus complexe géométriquement ?

geo["CP Ville"].loc[geo['merged_coordinates'].idxmax()]

'97370 MARIPASOULA'

In [135]:
#Quelles sont les villes qui sont de type "Polygon" mais dont la longueur des listes accédées avec la clé "coordinates" vaut 2 ?


geo["len_coordinates"] = geo["geo_shape_json"].apply(lambda x: len(x["coordinates"]))
geo.loc[(geo["type"] == "Polygon") & (geo["len_coordinates"] == 2)]


,Code Postal,Commune,Département,Région,Statut,Altitude Moyenne,Superficie,Population,geo_point_2d,geo_shape,Latitude,Longitude,CP Ville,geo_shape_json,type,coordinates,merged_coordinates,len_coordinates
Code INSEE,,,,,,,,,,,,,,,,,,
02232,02600,COYOLLES,AISNE,PICARDIE,Commune simple,126.0,2489.0,0.4,"49.2133089693, 3.01306173751","{""type"": ""Polygon"", ""coordinates"": [[[3.037897...",49.213309,3.013062,02600 COYOLLES,"{'type': 'Polygon', 'coordinates': [[[3.037897...",Polygon,"[[[3.037897560658703, 49.16921270775551], [3.0...",80,2
05052,05600,EYGLIERS,HAUTES-ALPES,PROVENCE-ALPES-COTE D'AZUR,Commune simple,1615.0,2987.0,0.7,"44.6925244966, 6.66688741371","{""type"": ""Polygon"", ""coordinates"": [[[6.719912...",44.692524,6.666887,05600 EYGLIERS,"{'type': 'Polygon', 'coordinates': [[[6.719912...",Polygon,"[[[6.719912606601575, 44.70392733592357], [6.7...",37,2
09042,09240,LA BASTIDE-DE-SEROU,ARIEGE,MIDI-PYRENEES,Chef-lieu canton,479.0,4138.0,1.0,"43.0242795037, 1.42677885373","{""type"": ""Polygon"", ""coordinates"": [[[1.423960...",43.024280,1.426779,09240 LA BASTIDE-DE-SEROU,"{'type': 'Polygon', 'coordinates': [[[1.423960...",Polygon,"[[[1.423960819701936, 42.99550854650971], [1.4...",46,2
22209,22650,PLOUBALAY,COTES-D'ARMOR,BRETAGNE,Chef-lieu canton,36.0,3600.0,2.7,"48.5596688192, -2.12838362694","{""type"": ""Polygon"", ""coordinates"": [[[-2.11237...",48.559669,-2.128384,22650 PLOUBALAY,"{'type': 'Polygon', 'coordinates': [[[-2.11237...",Polygon,"[[[-2.11237709725211, 48.52896113651634], [-2....",29,2
2A272,20100,SARTENE,CORSE-DU-SUD,CORSE,Sous-préfecture,190.0,20182.0,3.1,"41.5718443328, 8.93538107497","{""type"": ""Polygon"", ""coordinates"": [[[8.940487...",41.571844,8.935381,20100 SARTENE,"{'type': 'Polygon', 'coordinates': [[[8.940487...",Polygon,"[[[8.940487655060174, 41.490088793265336], [8....",152,2
2B049,20214,CALENZANA,HAUTE-CORSE,CORSE,Chef-lieu canton,596.0,18465.0,2.1,"42.4784267166, 8.80369225001","{""type"": ""Polygon"", ""coordinates"": [[[8.911220...",42.478427,8.803692,20214 CALENZANA,"{'type': 'Polygon', 'coordinates': [[[8.911220...",Polygon,"[[[8.911220447280732, 42.41019895400797], [8.8...",125,2
48080,48300,LANGOGNE,LOZERE,LANGUEDOC-ROUSSILLON,Chef-lieu canton,973.0,3124.0,3.1,"44.7237405653, 3.83363110341","{""type"": ""Polygon"", ""coordinates"": [[[3.850531...",44.723741,3.833631,48300 LANGOGNE,"{'type': 'Polygon', 'coordinates': [[[3.850531...",Polygon,"[[[3.850531831651621, 44.68745054452225], [3.8...",34,2
67486,67920,SUNDHOUSE,BAS-RHIN,ALSACE,Commune simple,166.0,1590.0,1.5,"48.2550723499, 7.63031906582","{""type"": ""Polygon"", ""coordinates"": [[[7.594939...",48.255072,7.630319,67920 SUNDHOUSE,"{'type': 'Polygon', 'coordinates': [[[7.594939...",Polygon,"[[[7.594939251325704, 48.23228567615893], [7.5...",43,2
68078,68420,EGUISHEIM,HAUT-RHIN,ALSACE,Commune simple,320.0,1400.0,1.6,"48.0371183049, 7.30052876111","{""type"": ""Polygon"", ""coordinates"": [[[7.281156...",48.037118,7.300529,68420 EGUISHEIM,"{'type': 'Polygon', 'coordinates': [[[7.281156...",Polygon,"[[[7.281156117554148, 48.02598189605906], [7.2...",54,2


In [108]:
#Pour ces villes vérifier que le premier polygone contient bien le second (enclave)
import shapely

shapely.geometry.polygon